In this notebook we explore the cold plasma dispersion relation and test a local implementation vs the one I made in PlasmaPy for testing purpose.

# The Cold Plasma Dielectric Tensor

Let's assume a magnetized plasma where the DC magnetific field is oriented along the $z$ direction: $\mathbf{B} = B_0 \mathbf{\hat{z}}$. In the cold plasma approximation, the dielectric tensor of the plasma can be written: 



$$
\mathbf{K}
=
\left(
\begin{array}{ccc}
S & j D & 0 \\
-j D & S & 0 \\
0 & 0 & P
\end{array}
\right)
$$
where
$$
\begin{eqnarray}
S &=& 1 - \sum_s \frac{\omega_{p,s}^2}{\omega^2 - \Omega_{c,s}^2} \\
D &=& \sum_s \frac{\Omega_{c,s}}{\omega} \frac{\omega_{p,s}^2}{\omega^2 - \Omega_{c,s}^2} \\
P &=& 1 - \sum_s \frac{\omega_{p,s}^2}{\omega^2}
\end{eqnarray}
$$

In [1]:
#%matplotlib widget

In [2]:
from scipy.constants import e, epsilon_0, pi, c, m_p

In [3]:
import sys
# import plasmapy, currently not in the python path
sys.path.append('../../PlasmaPy')
import plasmapy as pp
import astropy.units as u 
import numpy as np
from plasmapy.formulary import cold_plasma_permittivity_SDP
# from plasmapy.checks import check_quantity 
from plasmapy.formulary import gyrofrequency, plasma_frequency

In [4]:
B = 1 * u.T
n_e = 1.7e17 * u.m**-3
n_i = 1.7e17 * u.m**-3

omega_ce = gyrofrequency(B=B, particle='e', signed=True)
omega_ci = gyrofrequency(B=B, particle='D+', signed=True)
omega_pe = plasma_frequency(n_e, particle='e')
omega_pi = plasma_frequency(n_i, particle='D+')

In [5]:
# First try. 
def cold_plasma_tensor(n_e, n_i, omega, B, ions=['D+']):
    # electron contributions
    omega_ce = gyrofrequency(B, particle='e', signed=True)
    omega_pe = plasma_frequency(n_e, particle='e')
    S = 1 - omega_pe**2 / (omega**2 - omega_ce**2) 
    D =   omega_ce/omega * omega_pe**2 / (omega**2 - omega_ce**2)
    P = 1 - omega_pe**2 / omega**2
    print(omega_pe**2 / omega**2)
    # ion contributions
    for s in ions:
        omega_ci = gyrofrequency(B, particle=s, signed=True)
        omega_pi = plasma_frequency(n_i, particle=s)
        
        S -= omega_pi**2 / (omega**2 - omega_ci**2)
        D += omega_ci/omega * omega_pi**2 / (omega**2 - omega_ci**2)
        P -= omega_pi**2 / omega**2
        print(omega_ci**2 / omega**2)

    return S, D, P # S,D and P has no unit (to be checked)

In [6]:
# Same function, but shorter. Similar to PlasmaPy implementation
def cold_plasma_tensor2(n, omega, B, species=['e', 'D+']):
    S, D, P = 1, 0, 1 
    for s, n_s in zip(species, n):
        omega_c = gyrofrequency(B=B, particle=s, signed=True)
        omega_p = plasma_frequency(n=n_s, particle=s)
        
        S += - omega_p ** 2 / (omega ** 2 - omega_c ** 2)
        D += omega_c / omega * omega_p ** 2 / (omega ** 2 - omega_c ** 2)
        P += - omega_p ** 2 / omega ** 2
        print(omega_p**2 / omega**2)

    return S, D, P # S,D and P has no unit (to be checked)

In [7]:
Bs = np.array([1])*u.T
omega_RF = np.linspace(30,60,5)*1e6*(2*np.pi)*(u.rad/u.s)
print(omega_RF)

[1.88495559e+08 2.35619449e+08 2.82743339e+08 3.29867229e+08
 3.76991118e+08] rad / s


In [8]:
S, D, P = cold_plasma_tensor(n_e, n_i, omega=omega_RF, B=Bs, ions=['D+'])
print(S, D, P)

[15227.53958609  9745.6253351   6767.7953716   4972.25782403
  3806.88489652]
[0.06462404 0.04135939 0.0287218  0.02110173 0.01615601]
[-3.41778273 -1.75219692 -0.88087783 -0.36637224 -0.03670337] [17.44711915 13.61897345 11.20148627  9.52654334  8.29383122] [-15230.68823355  -9747.28046947  -6768.63921491  -4972.61248442
  -3806.92205839]


In [9]:
S, D, P = cold_plasma_tensor2(n=[n_e, n_i], omega=omega_RF, B=Bs, species=['e', 'D+'])
print(S, D, P)

[15227.53958609  9745.6253351   6767.7953716   4972.25782403
  3806.88489652]
[4.14864746 2.65513437 1.84384331 1.35466039 1.03716186]
[-3.41778273 -1.75219692 -0.88087783 -0.36637224 -0.03670337] [17.44711915 13.61897345 11.20148627  9.52654334  8.29383122] [-15230.68823355  -9747.28046947  -6768.63921491  -4972.61248442
  -3806.92205839]


In [10]:
S_, D_, P_ = cold_plasma_permittivity_SDP(B=Bs, species=['e', 'D+'], n=[n_e, n_i], omega=omega_RF)

In [11]:
np.isclose(S, S_)

array([ True,  True,  True,  True,  True])

In [12]:
np.isclose(D, D_)

array([ True,  True,  True,  True,  True])

In [13]:
np.isclose(P, P_)

array([ True,  True,  True,  True,  True])

In [14]:
%pylab
%matplotlib widget

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


C:\Users\JH218595\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\magics\pylab.py:160: UserWarning: pylab import has clobbered these variables: ['pi', 'e']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [15]:
from scipy.constants import epsilon_0, electron_mass, elementary_charge, physical_constants

# The Dispersion Relation
The _dispersion relation_ is the function that relates the frequency $\omega$ and the wavevector $k$. It characterizes each wave type and leads to the labels for the various type.


 - CMA diagram
 - phase velocity vs normalized frequency
 - normalized or not
 - density
 - angle
 - field strength
 
 - transverse motions of the electrons on cyclotron resonance sec.2.9.3

The plasma pulsation is :
$$
\omega_{p_s} = \sqrt{\frac{n_s q_s^2}{m_s \varepsilon_0}}
$$

In [16]:
def plasma_frequency(n, q, m):
    '''
    Returns the plasma angular frequency for a given species.
    '''
    omega_p = sqrt(n*q**2/(m*epsilon_0))
    return omega_p

def cyclotron_frequency(q, m, B0):
    '''
    Returns the cyclotron angular frequency for a given species.
    '''
    omega_c = np.abs(q)*B0/m
    return omega_c

Let's define a convenient object: a particle species.

In [17]:
class Species:
    def __init__(self, m, q, description=None):
        self.m = m
        self.q = q
        self.description = description
    def omega_p(self, n):
        return plasma_frequency(n, self.q, self.m)
    def omega_c(self, B0):
        return cyclotron_frequency(self.q, self.m, B0)
    def __repr__(self):
        return 'Specie:{}. Mass:{} kg, charge:{} C'.format(self.description, self.m, self.q)

In [18]:
electron = Species(electron_mass, -elementary_charge, description='Electron')
print(electron)

deuterium = Species(physical_constants['deuteron mass'][0], +elementary_charge, description='Deuterium')
print(deuterium)

Specie:Electron. Mass:9.10938356e-31 kg, charge:-1.6021766208e-19 C
Specie:Deuterium. Mass:3.343583719e-27 kg, charge:1.6021766208e-19 C


## The cold plasma tensor
The cold plasma tensor is given by:
$$
\mathbf{K} = \left(
\begin{matrix}
K_\perp & K_\times & 0 \\
-K_\times & K_\perp & 0 \\
0 & 0 & K_\parallel
\end{matrix}
\right)
$$
with
$$
\begin{array}{lcl}
K_\perp = S &=& 1 - \displaystyle \sum_k  \frac{\omega_{pk}^2}{\omega^2 - \omega_{ck}^2}
\\
i K_\times = D &=& \displaystyle \sum_k \frac{\epsilon_k \omega_{ck} \omega_{pk}^2}
{\omega \left( \omega^2 - \omega_{ck}^2\right)}
\\
K_\parallel = P &=& 1 - \displaystyle \sum_k \frac{\omega_{pk}^2}{\omega^2}
\end{array}
$$

In [19]:
def K_perp(species, n, B0, f):
    K_perp = 1
    omega = 2*np.pi*f

    for k, specie in enumerate(species):
        K_perp -= specie.omega_p(n[k])**2 / (omega**2 - specie.omega_c(B0)**2)        
    return K_perp

def K_parallel(species, n, f):
    K_parallel = 1
    omega = 2*np.pi*f
    for k,specie in enumerate(species):
        K_parallel -= specie.omega_p(n[k])**2 / omega**2
    return K_parallel

def K_cross(species, n, B0, f):
    K_cross = 0
    omega = 2*np.pi*f

    for k, specie in enumerate(species):
        K_cross +=  np.sign(specie.q) * specie.omega_c(B0) * specie.omega_p(n[k])**2 / (omega*(omega**2 - specie.omega_c(B0)**2))        
    return -1j*K_cross

In [20]:
plasma = (electron, deuterium)
n_e = 1e18 # m^-3
n_D = 1e18 # m^-3
n = (n_e, n_D)
B0 = 3 # T
f = 5e9 # Hz
print(K_perp(plasma, n, B0, f))
print(K_parallel(plasma, n, f))
print(K_cross(plasma, n, B0, f))
np.sign(electron.q)

1.0105934686511726
-2.225533974053524
-0.19268233325648582j


-1.0

In [44]:
freqs = np.logspace(6, 11, 1001)
plasma = (electron, deuterium)
n_e = 1e18 # m^-3
n_D = 1e18 # m^-3
n = (n_e, n_D)
B0 = 3 # T

fig, ax = plt.subplots(figsize=(10,4))

# # Manual way
# ax.loglog(freqs, K_perp(plasma, n, B0, freqs), lw=2, label='S (>0)')
# ax.loglog(freqs, - K_perp(plasma, n, B0, freqs), lw=2, label='S (<0)', color='C0', ls='--')
# ax.loglog(freqs, 1j*K_cross(plasma, n, B0, freqs), lw=2, label='jD (>0)', color='C1')
# ax.loglog(freqs, - 1j*K_cross(plasma, n, B0, freqs), lw=2, label='jD (<0)', color='C1', ls='--')
# ax.loglog(freqs, K_parallel(plasma, n, freqs), lw=2, label='P (>0)', color='C2')
# ax.loglog(freqs, - K_parallel(plasma, n, freqs), lw=2, label='P (<0)', color='C2', ls='--')

# PlasmaPy
S, D, P = cold_plasma_permittivity_SDP(B=B0*u.T, species=['e', 'D+'], n=[n_e/u.m**3, n_D/u.m**3], omega=2*pi*freqs*u.rad/u.s)
ax.loglog(freqs, S, lw=2, label='S (>0)')
ax.loglog(freqs, - S, lw=2, label='S (<0)', color='C0', ls='--')
ax.loglog(freqs, D, lw=2, label='jD (>0)', color='C1')
ax.loglog(freqs, - D, lw=2, label='jD (<0)', color='C1', ls='--')
ax.loglog(freqs, P, lw=2, label='P (>0)', color='C2')
ax.loglog(freqs, - P, lw=2, label='P (<0)', color='C2', ls='--')

ax.set_xlabel('f [Hz]', fontsize=16)
ax.grid(True, which='both', alpha=0.5)
ax.legend(fontsize=14, loc='upper right', ncol=3)

ax.axvline(deuterium.omega_c(B0)/(2*pi), lw=2, ls='--', color='k')
ax.text(x=9e6, y=1e4, s='$\Omega_{D}$', fontsize=16)
ax.axvline(deuterium.omega_p(n_e)/(2*pi), lw=2, ls='--', color='gray')
ax.text(x=1.7e8, y=1e2, s='$\omega_{D}$', fontsize=16)
ax.axvline(electron.omega_p(n_e)/(2*pi), lw=2, ls='--', color='gray')
ax.text(x=1e10, y=1e2, s='$\omega_{e}$', fontsize=16)
ax.axvline(electron.omega_c(B0)/(2*pi), lw=2, ls='--', color='k')
ax.text(x=5e10, y=1e3, s='$\Omega_{e}$', fontsize=16)
ax.set_xlim(1e6, 1e11)
ax.xaxis.set_tick_params(labelsize=16)
ax.yaxis.set_tick_params(labelsize=16)

ax.axvspan(30e6, 70e6, color='C3', alpha=0.3)
ax.text(x=35e6, y=1e3, s='ICRF', color='C3', fontsize=16)

ax.axvspan(1e9, 5e9, color='C4', alpha=0.3)
ax.text(x=1.5e9, y=1e3, s='LHRF', color='C4', fontsize=16)

ax.set_title(f'n={n_e}' + r'$m^{-3}$, $B_0$' + f'={B0}T')

fig.tight_layout()
fig.savefig('SDP_vs_f_nfixed_Bfixed.png', dpi=150)

# Solving the Dispersion Relation for a Tokamak

In [22]:
def solve_dispersion_relation(n, B, f=50e6, n_parallel=2):
    n_perps = []
        
    for _n, _B in zip(n,B):

        S, D, P = cold_plasma_permittivity_SDP(
                                                B=_B*u.T, 
                                               species=['e', 'D+'], 
                                               n=[_n/u.m**3, _n/u.m**3], 
                                               omega=2*pi*f*u.rad/u.s)
        S = S.value
        D = D.value
        P = P.value
        
        A = S
        B = -((S - n_parallel**2)*(S+P) - D**2)
        C = P*((S - n_parallel**2)**2 - D**2)
        p = (A,B,C)
        
        n_perp = np.roots(p)
    
        n_perps.append(n_perp)
    
    return np.array(n_perps)


def solve_dispersion_relation_ICRF(n, B, f=50e6, n_parallel=2):
    n_perps = []
        
    for _n, _B in zip(n,B):

        S, D, P = cold_plasma_permittivity_SDP(
                                                B=_B*u.T, 
                                               species=['e', 'D+'], 
                                               n=[_n/u.m**3, _n/u.m**3], 
                                               omega=2*pi*f*u.rad/u.s)
        S = S.value
        D = D.value
        P = P.value
        R = S + D
        L = S - D 
        
        n_perp_squared_FW = (R - n_parallel**2)*(L - n_parallel**2)/(S - n_parallel**2)
        n_perp_squared_SW = P*(1 - n_parallel**2/S)
        n_perps.append([n_perp_squared_SW, n_perp_squared_FW])
    
    return np.array(n_perps)

def solve_dispersion_relation_LHRF(n, B, f=3.7e9, n_parallel=2):
    n_perps = []
        
    for _n, _B in zip(n,B):

        S, D, P = cold_plasma_permittivity_SDP(
                                                B=_B*u.T, 
                                               species=['e', 'D+'], 
                                               n=[_n/u.m**3, _n/u.m**3], 
                                               omega=2*pi*f*u.rad/u.s)
        S = S.value
        D = D.value
        P = P.value
        
        n_perp_squared_FW =  (S - n_parallel**2)
        n_perp_squared_SW = P/S*(S - n_parallel**2)
        n_perps.append([n_perp_squared_SW, n_perp_squared_FW])
    
    return np.array(n_perps)

In [23]:
def density_profile(rho, n_avg=1e19, nu_n=-2):
    """
    rho = 0 at magnetic axis and =1 at outer plasma surface
    n_avg is volume averaged density
    nu_n : peaking factors
    """
    return n_avg*(1 + rho**2)**nu_n

def B_profile(R, B0=3.7, R0=2.5):
    return B0*R0/R

# Radial Scan

In [24]:
R0 = 2.5  # m
a = 0.5  # m
rho = np.linspace(0.8,1.0, 101)
R = R0 + rho*a
ne = density_profile(rho)
B = B_profile(R)

n_perp_square = np.array(solve_dispersion_relation(ne, B, f=50e6))

R_SOL = R0 + a*np.linspace(1,1.3, 201)
ne_SOL = ne[-1] * np.exp( - (R_SOL - (R0+a))/0.1)
B_SOL = B_profile(R_SOL)
n_perp_square_SOL = solve_dispersion_relation(ne_SOL, B_SOL, f=55e6, n_parallel=7)


In [25]:
fig, ax = plt.subplots(4, 1, sharex=True, figsize=(10,4))
ax[0].plot(R, B, color='C0', lw=2)
ax[0].plot(R_SOL, B_SOL, color='C0', lw=2)

ax[1].plot(R, ne, color='C1', lw=2)
ax[1].plot(R_SOL, ne_SOL, color='C1', lw=2)
# ax[1].set_yscale('log')

n_SW = n_perp_square[:,0]
n_FW = n_perp_square[:,1] 
n_SW_SOL = n_perp_square_SOL[:,0]
n_FW_SOL = n_perp_square_SOL[:,1] 

ax[2].plot(R, np.sign(n_SW)*np.log(np.abs(n_SW)), color='C1' )
ax[2].plot(R_SOL, np.sign(n_SW_SOL)*np.log(np.abs(n_SW_SOL)), color='C1')

ax[3].plot(R, np.sign(n_FW)*np.log(np.abs(n_FW)), color='C2' )
ax[3].plot(R_SOL, np.sign(n_FW_SOL)*np.log(np.abs(n_FW_SOL)), color='C2')


ax[-1].set_xlabel('R [m]')
ax[0].text(x=3.01, y=3.5, s='LCFS')
ax[0].text(x=3.11, y=3.5, s='Antenna')


fig.subplots_adjust(hspace=0)
for _a in ax:
    _a.axvline(R0+a, ls='--', color='gray')
    _a.axvline(3.1, ls='--', color='gray')

# Density Scan ICRF

In [26]:
ne = np.logspace(16, 20, 201)

In [27]:
B0 = 3.0
B = np.full_like(ne, B0) 
f = 55e6
k0 = 2*np.pi/c*f
nz_dipole  = 9 / k0

In [28]:
n_perp_square = solve_dispersion_relation(ne, B, f, n_parallel=5)
n_perp_square1 = solve_dispersion_relation(ne, B, f, n_parallel=5)
n_perp_square2 = solve_dispersion_relation(ne, B, f, n_parallel=15)
# n_perp_square_ICRF = solve_dispersion_relation_ICRF(ne, B, f, n_parallel=nz_dipole)

In [29]:
# cut-off density
S,D,P = cold_plasma_permittivity_SDP(B*u.T, species=('e-', 'D+'), n=(ne/u.m**3, ne/u.m**3), omega=2*pi*f*u.rad/u.s)

In [30]:
R = S + D
L = S- D

In [31]:
nc = ne[np.argmin(np.abs(5**2 - R))]

In [32]:
fig, ax = plt.subplots(figsize=(8,4))
ax.plot(ne[S>0],  np.abs(S[S>0]), lw=2, color='C0', label='S > 0')
ax.plot(ne[S<0],  np.abs(S[S<0]), lw=2, color='C0', ls='--', label='S < 0')
ax.plot(ne[D>0],  np.abs(D[D>0]), lw=2, color='C1', label='D > 0')
ax.plot(ne[D<0],  np.abs(D[D<0]), lw=2, color='C1', ls='--', label='D < 0')
ax.plot(ne[P>0],  np.abs(P[P>0]), lw=2, color='C2', label='P > 0')
ax.plot(ne[P<0],  np.abs(P[P<0]), lw=2, color='C2', ls='--', label='P < 0')

ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlabel('Electron Density $n_e$ [$m^{-3}$]', fontsize=16)
ax.set_ylabel(r'$ \left| X \right|$', fontsize=16)
# ax.plot(ne, np.log(k0*np.abs(n_perp_square_ICRF))*np.sign(n_perp_square_ICRF))
ax.legend( fontsize=14, ncol=3, loc='upper right')
ax.grid(True, which='both', alpha=0.5)
ax.tick_params(axis='both', which='major', labelsize=16)
ax.set_xlim(1e20, 1e16)

ax.text(5e19, 0.10, 'core plasma')
ax.text(2.5e18, 0.10, 'edge plasma')
ax.text(0.5e17, 0.10, 'SOL')


fig.tight_layout()
fig.savefig('IC_SDP.png', dpi=150)

In [33]:
fig, ax = plt.subplots(figsize=(8,4))

ax.fill_between(ne, np.log(np.abs(n_perp_square1[:,0]))*np.sign(n_perp_square1[:,0]), 
                              np.log(np.abs(n_perp_square2[:,0]))*np.sign(n_perp_square2[:,0]), alpha=0.5, color='C0')
ax.fill_between(ne, np.log(np.abs(n_perp_square1[:,1]))*np.sign(n_perp_square1[:,1]), 
                              np.log(np.abs(n_perp_square2[:,1]))*np.sign(n_perp_square2[:,1]), alpha=0.5, color='C1')

ax.plot(ne, np.log(np.abs(n_perp_square1[:,0]))*np.sign(n_perp_square1[:,0]), lw=3, color='lightblue')
ax.plot(ne, np.log(np.abs(n_perp_square1[:,1]))*np.sign(n_perp_square1[:,1]), lw=3, color='lightcoral')
ax.plot(ne, np.log(np.abs(n_perp_square2[:,0]))*np.sign(n_perp_square2[:,0]), lw=3, color='darkblue')
ax.plot(ne, np.log(np.abs(n_perp_square2[:,1]))*np.sign(n_perp_square2[:,1]), lw=3, color='darkred')


# n_S = n_perp_square[:,0]
# n_F = n_perp_square[:,1]

# ax.fill_between(ne, np.abs(n_perp_square1[:,0]), 
#                               np.abs(n_perp_square2[:,0]), alpha=0.5, color='C0' ) 
# ax.fill_between(ne, np.abs(n_perp_square1[:,1]), 
#                               np.abs(n_perp_square2[:,1]), alpha=0.5, color='C1' ) 

# ax.plot(ne[n_S>0], (np.abs(n_S[n_S>0])), lw=3, color='C0', label=r'$n_{\perp,S}^2$ ($>0$)')
# ax.plot(ne[n_S<0], (np.abs(n_S[n_S<0])), lw=3, ls='--', color='C0', label=r'$n_{\perp,S}^2$ ($<0$)')

# ax.plot(ne[n_F>0], (np.abs(n_F[n_F>0])), lw=3, color='C1', label=r'$n_{\perp,F}^2$ ($>0$)')
# ax.plot(ne[n_F<0], (np.abs(n_F[n_F<0])), lw=3, ls='--', color='C1', label=r'$n_{\perp,F}^2$ ($<0$)')
# ax.set_yscale('log')

ax.set_xlabel('Electron Density $n_e$ [$m^{-3}$]', fontsize=16)
ax.set_ylabel(r'$\mathrm{sign}\left( n^2_\perp \right) \log \left| n^2_\perp \right|$', fontsize=16)
# ax.plot(ne, np.log(k0*np.abs(n_perp_square_ICRF))*np.sign(n_perp_square_ICRF))
ax.legend(('SW', 'FW'), fontsize=16)
ax.axhspan(-17, 0, color='gray', alpha=0.2)
ax.set_xscale('log')
ax.grid(True, which='both', alpha=0.5)
ax.tick_params(axis='both', which='major', labelsize=16)
# ax.set_ylim(-16, 16)
ax.set_xlim(1e20, 1e16)

ax.text(5e16, 3, 'Progagative')
ax.text(5e16, -10, 'Evanescent')

ax.annotate("$n_\parallel=5$", xy=(1e19, 5), xytext=(2e19, 10), arrowprops=dict(arrowstyle="->", color='coral'))
ax.annotate("$n_\parallel=15$", xy=(2e19, 1), xytext=(7e19, 1), arrowprops=dict(arrowstyle="->", color='darkred'))

ax.annotate("$n_\parallel=5$", xy=(1e18, -12), xytext=(3e17, -8), arrowprops=dict(arrowstyle="->", color='blue'))
ax.annotate("$n_\parallel=15$", xy=(2e18, -13.5), xytext=(1e19, -8), arrowprops=dict(arrowstyle="->", color='darkblue'))

ax.text(5e19, 20, 'core plasma')
ax.text(2.5e18, 20, 'edge plasma')
ax.text(1e17, 20, 'SOL')

#ax.axvline(nc)

fig.tight_layout()
fig.savefig('n_perp_square_vs_ne_ICRF.png', dpi=150)

# Density Scan LHRF

In [34]:
ne = np.logspace(16, 19.5, 301)

In [35]:
B0 = 3
B = np.full_like(ne, B0) 
f = 3.7e9
k0 = 2*np.pi/c*f

In [36]:
# cut-off density
S,D,P = cold_plasma_permittivity_SDP(B*u.T, species=('e-', 'D+'), n=(ne/u.m**3, ne/u.m**3), omega=2*pi*f*u.rad/u.s)

In [37]:
fig, ax = plt.subplots(figsize=(8,4))
ax.plot(ne[S>0],  np.abs(S[S>0]), lw=2, color='C0', label='S > 0')
ax.plot(ne[S<0],  np.abs(S[S<0]), lw=2, color='C0', ls='--', label='S < 0')
ax.plot(ne[D>0],  np.abs(D[D>0]), lw=2, color='C1', label='D > 0')
ax.plot(ne[D<0],  np.abs(D[D<0]), lw=2, color='C1', ls='--', label='D < 0')
ax.plot(ne[P>0],  np.abs(P[P>0]), lw=2, color='C2', label='P > 0')
ax.plot(ne[P<0],  np.abs(P[P<0]), lw=2, color='C2', ls='--', label='P < 0')

ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlabel('Electron Density $n_e$ [$m^{-3}$]', fontsize=16)
ax.set_ylabel(r'$ \left| X \right|$', fontsize=16)
# ax.plot(ne, np.log(k0*np.abs(n_perp_square_ICRF))*np.sign(n_perp_square_ICRF))
ax.legend( fontsize=14, ncol=3, loc='upper right')
ax.grid(True, which='both', alpha=0.5)
ax.tick_params(axis='both', which='major', labelsize=16)
ax.set_xlim(1e20, 1e16)

ax.text(5e19, 0.0050, 'core plasma')
ax.text(2.5e18, 0.0050, 'edge plasma')
ax.text(0.5e17, 0.0050, 'SOL')


fig.tight_layout()
fig.savefig('LH_SDP.png', dpi=150)

In [38]:
n_perp_square = solve_dispersion_relation(ne, B, f, n_parallel=2)
n_perp_square_LHRF = solve_dispersion_relation_LHRF(ne, B, f, n_parallel=2)

In [39]:
fig, ax = plt.subplots(figsize=(8,4))


n_S = n_perp_square_LHRF[:,0]
n_F = n_perp_square_LHRF[:,1]

# ax.plot(ne, np.log(np.abs(n_perp_square1[:,0]))*np.sign(n_perp_square1[:,0]), lw=3, color='lightblue')
# ax.plot(ne, np.log(np.abs(n_perp_square1[:,1]))*np.sign(n_perp_square1[:,1]), lw=3, color='lightcoral')
# ax.plot(ne, np.log(np.abs(n_perp_square2[:,0]))*np.sign(n_perp_square2[:,0]), lw=3, color='darkblue')
# ax.plot(ne, np.log(np.abs(n_perp_square2[:,1]))*np.sign(n_perp_square2[:,1]), lw=3, color='darkred')
# ax.plot(ne, np.log(np.abs(n_perp_square))*np.sign(n_perp_square), lw=3)
# ax.plot(ne, np.sign(n_perp_square_LHRF)*np.log(np.abs(n_perp_square_LHRF)), lw=3, ls='-')

ax.plot(ne[n_S>0], (np.abs(n_S[n_S>0])), lw=3, color='C0', label=r'$n_{\perp,S}^2$ ($>0$)')
ax.plot(ne[n_S<0], (np.abs(n_S[n_S<0])), lw=3, ls='--', color='C0', label=r'$n_{\perp,S}^2$ ($<0$)')

ax.plot(ne[n_F>0], (np.abs(n_F[n_F>0])), lw=3, color='C1', label=r'$n_{\perp,F}^2$ ($>0$)')
ax.plot(ne[n_F<0], (np.abs(n_F[n_F<0])), lw=3, ls='--', color='C1', label=r'$n_{\perp,F}^2$ ($<0$)')

ax.set_xlabel('Electron Density $n_e$ [$m^{-3}$]', fontsize=16)
# ax.set_ylabel(r'$\mathrm{sign}\left( n^2_\perp \right) \log \left| n^2_\perp \right|$', fontsize=16)
ax.set_ylabel(r'$ \left| n^2_\perp \right|$', fontsize=16)
# ax.plot(ne, np.log(k0*np.abs(n_perp_square_ICRF))*np.sign(n_perp_square_ICRF))
ax.legend(fontsize=14, ncol=2, loc='upper right')
# ax.axhspan(-17, 0, color='gray', alpha=0.2)
ax.set_xscale('log')
ax.grid(True, which='both', alpha=0.5)
ax.tick_params(axis='both', which='major', labelsize=16)
# ax.set_ylim(-3, 5)
ax.set_xlim(3e19, 1e16)

# ax.text(5e16, 1, 'Progagative')
# ax.text(5e16, -1, 'Evanescent')

# ax.annotate("$n_\parallel=5$", xy=(1e19, 5), xytext=(2e19, 10), arrowprops=dict(arrowstyle="->", color='coral'))
# ax.annotate("$n_\parallel=15$", xy=(2e19, 1), xytext=(7e19, 1), arrowprops=dict(arrowstyle="->", color='darkred'))

# ax.annotate("$n_\parallel=5$", xy=(1e18, -12), xytext=(3e17, -8), arrowprops=dict(arrowstyle="->", color='blue'))
# ax.annotate("$n_\parallel=15$", xy=(2e18, -13.5), xytext=(1e19, -8), arrowprops=dict(arrowstyle="->", color='darkblue'))
ax.set_yscale('log')
ax.text(3e19, 0.05, 'core plasma')
ax.text(2.5e18, 0.05, 'edge plasma')
ax.text(1e17, 0.05, 'SOL')

ax.axvline(0.0124*3.7e9**2, color='gray', ls='--')
ax.annotate("$n_c$", xy=(1.7e17, 10), xytext=(1e17, 10), arrowprops=dict(arrowstyle="->"))

fig.tight_layout()
fig.savefig('n_perp_square_vs_ne_LHRF.png', dpi=150)

# References
 - Swanson, Plasma Waves, chap.2